<a href="https://colab.research.google.com/github/mcaires2/ResumoTextoSpacyExtraction/blob/master/MachineLearning_Spacy_Summarization_001_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m spacy download pt_core_news_sm

In [1]:
import spacy
from pathlib import Path
nlp= spacy.load('pt_core_news_sm')

In [2]:
from spacy.lang.pt.stop_words import STOP_WORDS
stop_words=list(STOP_WORDS)
stopWords =['arq', 'identificador', '419', 'ref', 'validação', '2006', '2001', '200', 'jus', '0014', 'mov', 'rua', 'resolução', 'https', 'das', 'deste', 'processo', 'lei', 'conforme', 'autora', 'dos', 'inicial', 'petição', 'não', 'para', 'com', 'por', 'projudi', 'que']
stopWords2 =['documento','tjpr','juntada','assinado','digitalmente','petição','inicial','como','seus','pelo','pela','também','todos','todas','pois','ação','deve','parte','seja']
stopWords3 =['assim','pelos','ainda','prova','código','mais','caso','demanda','são','nos','seu','sem','aos','uma','ser','sua','cep','pr','nº']
stop_words = stopWords + stopWords2 +stopWords3 + stop_words
#print(stop_words)

In [25]:
ArquivoCaminhoAbsoluto =Path(r"/content/drive/My Drive/Colab Notebooks/MachineLearning/MachineLearningLDA/ExtravioBagagem/DadosTXT/ModeloLimpo.txt")
ArquivoDiretorioResumoDestino=Path(r"/content/drive/My Drive/Colab Notebooks/MachineLearning/ResumosMachineLearning")
ArquivoNomePuro =Path(ArquivoCaminhoAbsoluto).stem

def extrair_txt(filename):
    
    #with open(filename, 'r', encoding = "ISO-8859-1") as f:
    with open(filename,'r',encoding='utf-8')as f: # se extrair o txt direto pelo word ou em sites online use esta linha, se o txt foi extraído de pdf usar o encoding de cima...
       temp_text = f.read()
              
    return temp_text

texto = extrair_txt(ArquivoCaminhoAbsoluto)
doc=nlp(texto)
corpus = [sent.text.lower() for sent in doc.sents ]

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words=stop_words)   
cv_fit=cv.fit_transform(corpus)    
word_list = cv.get_feature_names();    
count_list = cv_fit.toarray().sum(axis=0) # contar quantas vezes cada palavra que não caiu na lista de irrelevância aparece no corpus documento (já separado por sentenças)
word_frequency = dict(zip(word_list,count_list)) # dicionario formado por cada palavra e nr vezes que aparece ex {'00': 11, '000': 13, '0001': 1, '0009673': 21...} pair kv

In [8]:
# print(word_list)
# print(count_list)
# print(word_frequency)

In [27]:
val=sorted(word_frequency.values())
higher_word_frequencies = [word for word,freq in word_frequency.items() if freq in val[-5:]]
# print("\nWords with higher frequencies: ", higher_word_frequencies)

In [28]:
# gets relative frequency of words
# retiro o nr de vezes que uma palavra apareceu e este passa a ser o valor 1 divisor e todas as outras terão o valor delas dividido por este numero ex 10/52 = 0.19230769
higher_frequency = val[-1] #52

for word in word_frequency.keys():  
    word_frequency[word] = (word_frequency[word]/higher_frequency) 

In [29]:
#
# Rank de Sentenças -> com os valore relativos de cada palavra vou encontrar o valor da sentença -> soma dos valores relativos de cada uma das palavras que a compõe
#


sentence_rank={}
for sent in doc.sents:
    for word in sent :       
        if word.text.lower() in word_frequency.keys():            
            if sent in sentence_rank.keys():
                sentence_rank[sent]+=word_frequency[word.text.lower()]
            else:
                sentence_rank[sent]=word_frequency[word.text.lower()]


#print(sentence_rank)


In [30]:

#
#Esta parte é interessante porque faço um rank matemático de sentenças no que tinge a soma dos valores relativos de suas palavras mas não modifico a ordem que elas aparecem no texto
#com isso supero questão de desorganizar a narrativa.
#[12.249999999999998, 8.192307692307692, 7.557692307692306, 7.499999999999997, 7.288461538461538, 7.153846153846152, 6.711538461538462, 6.269230769230768, 6.211538461538461, 6.153846153846154]


top_sentences=(sorted(sentence_rank.values())[::-1]) #- ordem decrescente
top_sent=top_sentences[:5]
# print(top_sent)

In [31]:
texto_resumido=[]
for sent,strength in sentence_rank.items():  
    if strength in top_sent: # se a soma relativa das palavras de uma sentença estiver no top_sent
        texto_resumido.append(sent)
    else:
        continue
for item in texto_resumido:
    print(item,end=" ")

Contudo, os autores sofreram com a desorganização e o péssimo serviço prestado pela companhia aérea ré, vejamos: no dia 23/12/2019, por se tratar de um trecho internacional, o casal de autores chegou ao aeroporto de Lisboa (PT) às 16h, ou seja, duas horas antes do horário de saída do voo. Porém, os autores novamente foram surpreendidos, uma vez que visualizaram no painel de voos do aeroporto, que o único voo atrasado era o da companhia ré, responsável pelo trecho Lisboa (PT) – Paris (FR) - VOO TP 442 -, sendo que o novo horário de saída já estava para às 11h55min. Inconformados, o casal de autores e mais alguns brasileiros que também receberam essa informação, até argumentaram que as bagagens estavam dentro dos limites de peso para serem consideradas bagagens de mão, sendo que até se propuseram a pesar as bagagens para provarem que estavam dentro dos padrões, porém a companhia aérea ré apenas comunicou que não tinha o medidor adequado no local e que se os autores não despachassem as ba

In [33]:
#
#Gerando Documento Word com o Texto de Resumo
#

!pip install python-docx
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
import os

ArquivoDiretorioResumoDestino=Path(r"/content/drive/My Drive/Colab Notebooks/MachineLearning/ResumosMachineLearning")
os.chdir(ArquivoDiretorioResumoDestino)

# quebras de sentenças por paragráfos
document = Document()
NomeArquivoSaida = 'Resumo ' + ArquivoNomePuro + '.doc'
document.add_heading(f'ResumoGeradoExtração - {NomeArquivoSaida}', 0)

p = document.add_paragraph(' ')
p = document.add_paragraph(' ')

contador = 0
for item in texto_resumido:
  p = document.add_paragraph('Trecho ' + str(contador) + ': ' + str(item))
  p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  contador = contador +1


p = document.add_paragraph(' ')
p = document.add_paragraph(' ')

p = document.add_paragraph(' ')
p.add_run('ResumoGeradoExtração Continua ').underline = True
p = document.add_paragraph(' ')


t_continua =''
for item in texto_resumido:
    t_continua = t_continua + ' ' + str(item)
p = document.add_paragraph(t_continua)
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY



document.save(NomeArquivoSaida)
print(' (: ')
print('Arquivo WORD.DOC  Gerado com Sucesso')

 (: 
Arquivo WORD.DOC  Gerado com Sucesso
